# Preprocessing

In [2]:
import pandas as pd
import json
import numpy as np

# Function to load JSON file into Pandas DataFrame
def load_json(filename):
    """Loads a JSON file into a pandas DataFrame"""
    data = []
    with open(filename, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    return pd.DataFrame(data)

# Load Yelp dataset
business_df = load_json(r"C:\BIANCONERI\Master's AI SJSU\5- Advanced Data Mining\Final Project\Yelp-JSON\Yelp JSON\yelp_dataset\yelp_academic_dataset_business.json")
review_df = load_json(r"C:\BIANCONERI\Master's AI SJSU\5- Advanced Data Mining\Final Project\Yelp-JSON\Yelp JSON\yelp_dataset\yelp_academic_dataset_review.json")
user_df = load_json(r"C:\BIANCONERI\Master's AI SJSU\5- Advanced Data Mining\Final Project\Yelp-JSON\Yelp JSON\yelp_dataset\yelp_academic_dataset_user.json")
print(f"business_df {business_df.shape}")
print(f"review_df {review_df.shape}")
print(f"user_df {user_df.shape}")

# Filter only OPEN businesses in CALIFORNIA W/ 'categories' column exists and is not null
business_df = business_df[(business_df['state'] == 'CA') & (business_df['is_open'] == 1) & (business_df['categories'].notna())]
print(f"business_df {business_df.shape}")
print(business_df.head())

business_df (150346, 14)
review_df (6990280, 9)
user_df (1987897, 22)
business_df (4064, 14)
                business_id                             name  \
26   noByYNtDLQAra9ccqxdfDw                              H&M   
85   IDtLPgUrqorrpqSLdfMhZQ             Helena Avenue Bakery   
91   nUqrF-h9S7myCcvNDecOvw             Iron Horse Auto Body   
120  bYjnX_J1bHZob10DoSFkqQ      Tinkle Belle Diaper Service   
141  SZU9c8V2GuREDN5KgyHFJw  Santa Barbara Shellfish Company   

                   address           city state postal_code   latitude  \
26        827-833 State St  Santa Barbara    CA       93101  34.420209   
85   131 Anacapa St, Ste C  Santa Barbara    CA       93101  34.414445   
91          825 Cacique St  Santa Barbara    CA       93103  34.419620   
120                         Santa Barbara    CA       93101  34.420334   
141      230 Stearns Wharf  Santa Barbara    CA       93101  34.408715   

      longitude  stars  review_count  is_open  \
26  -119.700460    3.0      

In [3]:
# Extract and print all unique categories from business_df
unique_categories = set()
business_df['categories'].apply(lambda x: unique_categories.update(x.split(', ')))
print(f"Number of unique categories: {len(unique_categories)}")
print(unique_categories)

Number of unique categories: 946
{'Public Transportation', 'Batting Cages', 'Crane Services', 'Massage Schools', 'Cuban', 'Solar Installation', 'Tui Na', 'Greek', 'Tree Services', 'Private Investigation', 'Party Bus Rentals', 'Obstetricians & Gynecologists', 'Orthopedists', 'Patisserie/Cake Shop', 'Beer Gardens', 'Acne Treatment', 'Naturopathic/Holistic', 'General Litigation', 'Banks & Credit Unions', 'Preventive Medicine', 'Saunas', 'Dermatologists', 'International Grocery', 'Firewood', 'Arts & Crafts', 'Montessori Schools', 'Oncologist', 'Ear Nose & Throat', 'Building Supplies', 'Spiritual Shop', 'Auction Houses', 'Mortuary Services', 'Child Care & Day Care', 'Home Network Installation', 'Couriers & Delivery Services', 'Audio/Visual Equipment Rental', 'Party Equipment Rentals', 'Counseling & Mental Health', 'Food Delivery Services', 'Software Development', 'Rafting/Kayaking', 'Wheel & Rim Repair', 'Pool & Hot Tub Service', 'Beer', 'Investing', 'Olive Oil', 'Pet Groomers', 'Utilities'

In [4]:
# Convert categories to lowercase for consistency
business_df['categories'] = business_df['categories'].apply(lambda x: [category.lower() for category in x.split(', ')])

In [5]:
# Define restaurant-related keywords
restaurant_keywords = ["bars","donuts","barbeque", "sandwiches","wineries","fish & chips","vegetarian", "beer", "food", "dessert", "gelato", "restaurants", "wine", "tacos", "tea", "acai bowls", "whiskey", "juice bars & smoothies", "poke", "spirits", "cocktail", "salad", "coffee", "bakeries", "breweries", "pizza", "burgers", "soup", "bagels", "ice cream & frozen yogurt", "ramen", "chicken wings", "food trucks", "cafes", "seafood", "vegan", "diners", "noodles"]

# Filter RESTAURANTS ONLY (businesses with restaurant-related keywords)
restaurants_df = business_df[business_df['categories'].apply(lambda x: any(keyword in x for keyword in restaurant_keywords))]

print(f"restaurants_df {restaurants_df.shape}")

# Select relevant columns // we may get rid of address related columns if we won't use them
restaurants_df = restaurants_df[['business_id', 'name', 'address', 'city', 'postal_code', 'latitude', 'longitude', 'categories', 'stars', 'review_count']]

print(f"restaurants_df {restaurants_df.shape}")
print(restaurants_df.head())

# Drop rows with missing values // there is none
#df_clean = restaurants_df.dropna()
#print(f"df_clean {df_clean.shape}")

restaurants_df (1015, 14)
restaurants_df (1015, 10)
                business_id                             name  \
85   IDtLPgUrqorrpqSLdfMhZQ             Helena Avenue Bakery   
141  SZU9c8V2GuREDN5KgyHFJw  Santa Barbara Shellfish Company   
431  ifjluUv4VASwmFqEp8cWlQ                    Marty's Pizza   
470  VeFfrEZ4iWaecrQg6Eq4cg                         Cal Taco   
555  bdfZdB2MTXlT6-RBjSIpQg                       Pho Bistro   

                       address           city postal_code   latitude  \
85       131 Anacapa St, Ste C  Santa Barbara       93101  34.414445   
141          230 Stearns Wharf  Santa Barbara       93101  34.408715   
431         2733 De La Vina St  Santa Barbara       93105  34.436236   
470  7320 Hollister Ave, Ste 1         Goleta       93117  34.430542   
555  903 Embarcadero Del Norte     Isla Vista       93117  34.412934   

      longitude                                         categories  stars  \
85  -119.690672  [food, restaurants, salad, coffee & 

In [6]:
# Select relevant columns from review_df // I dropped review_id, date, useful, funny, cool
review_df = review_df[['user_id','business_id','stars','text']]
print(review_df.head())

# Merge reviews with restaurant using business_id as a key
merged_df = review_df.merge(restaurants_df[['business_id', 'name', 'categories']], on='business_id')
print(merged_df.head())

                  user_id             business_id  stars  \
0  mh_-eMZ6K5RLWhZyISBhwA  XQfwVwDr-v0ZS3_CbbE5Xw    3.0   
1  OyoGAe7OKpv6SyGZT5g77Q  7ATYjTIgM3jUlt4UM3IypQ    5.0   
2  8g_iMtfSiwikVnbP2etR0A  YjUWPpI6HXG530lwP-fb2A    3.0   
3  _7bHUi9Uuf5__HHc_Q8guQ  kxX2SOes4o-D3ZQBkiMRfA    5.0   
4  bcjbaE6dDog4jkNY91ncLQ  e4Vwtrqf-wpJfwesgvdgxQ    4.0   

                                                text  
0  If you decide to eat here, just be aware it is...  
1  I've taken a lot of spin classes over the year...  
2  Family diner. Had the buffet. Eclectic assortm...  
3  Wow!  Yummy, different,  delicious.   Our favo...  
4  Cute interior and owner (?) gave us tour of up...  
                  user_id             business_id  stars  \
0  59MxRhNVhU9MYndMkz0wtw  gebiRewfieSdtt17PTW6Zg    3.0   
1  OhECKhQEexFypOMY6kypRw  vC2qm1y3Au5czBtbhc-DNw    4.0   
2  4hBhtCSgoxkrFgHa4YAD-w  bbEXAEFr4RYHLlZ-HFssTA    5.0   
3  bFPdtzu11Oi0f92EAcjqmg  IDtLPgUrqorrpqSLdfMhZQ    5.0   
4  JYYYKt

In [7]:
# Fix category column if it's a list -> string
merged_df['categories'] = merged_df['categories'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
print(merged_df.head())

                  user_id             business_id  stars  \
0  59MxRhNVhU9MYndMkz0wtw  gebiRewfieSdtt17PTW6Zg    3.0   
1  OhECKhQEexFypOMY6kypRw  vC2qm1y3Au5czBtbhc-DNw    4.0   
2  4hBhtCSgoxkrFgHa4YAD-w  bbEXAEFr4RYHLlZ-HFssTA    5.0   
3  bFPdtzu11Oi0f92EAcjqmg  IDtLPgUrqorrpqSLdfMhZQ    5.0   
4  JYYYKt6TdVA4ng9lLcXt_g  SZU9c8V2GuREDN5KgyHFJw    5.0   

                                                text  \
0  Had a party of 6 here for hibachi. Our waitres...   
1  Yes, this is the only sushi place in town. How...   
2  Great burgers,fries and salad!  Burgers have a...   
3  What a great addition to the Funk Zone!  Grab ...   
4  We were a bit weary about trying the Shellfish...   

                              name  \
0  Hibachi Steak House & Sushi Bar   
1                       Sushi Teri   
2  The Original Habit Burger Grill   
3             Helena Avenue Bakery   
4  Santa Barbara Shellfish Company   

                                          categories  
0     steakhouses,

# Feature Extraction & Similarity Computation

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Feature Extraction for Content-Based Filtering: (TF-IDF) on categories and reviews
tfidf_category = TfidfVectorizer(stop_words='english')
tfidf_review = TfidfVectorizer(stop_words='english')
category_matrix = tfidf_category.fit_transform(merged_df['categories'])
review_matrix = tfidf_review.fit_transform(merged_df['text'])

In [9]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute similarity matrices
category_similarity = cosine_similarity(category_matrix)
review_similarity = cosine_similarity(review_matrix)

# Combine the two similarity matrices into a total similarity matrix by taking their average: 
# Equal weight to both categories and reviews when computing the overall similarity between restaurants.
total_similarity = (category_similarity + review_similarity) / 2

MemoryError: Unable to allocate 242. GiB for an array with shape (32478999457,) and data type int64

In [10]:
from sklearn.neighbors import NearestNeighbors

# Switched to use NearestNeighbors for memory-efficient similarity computation
category_nn = NearestNeighbors(metric='cosine').fit(category_matrix)
review_nn = NearestNeighbors(metric='cosine').fit(review_matrix)